# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Get the cities saved in the previous jupyter notebook
cities_data = pd.read_csv("../output_data/cities.csv")
cities_data.dropna(inplace = True) 
cities_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.12,-134.97,19.43,59,100,6.22,PF,1595453499
1,busselton,-33.65,115.33,13.89,53,37,9.34,AU,1595453423
2,jamestown,42.10,-79.24,26.67,73,20,5.70,US,1595453169
3,bengkulu,-3.80,102.27,23.30,88,90,1.98,ID,1595453499
4,bowen,-20.02,148.23,17.22,93,90,2.10,AU,1595453499


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Confgure the map using humidity as the weight
gmaps.configure(api_key=g_key)
locations = cities_data[["Lat", "Lng"]]
humidity = cities_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(zoom_level=2, center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add the layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Only pick the cities that have temperature between 21 and 26 degrees celsius, low wind speed, no clouds
cities_data = cities_data.loc[(cities_data['Max Temp'] >= 21) & (cities_data['Max Temp'] <= 27) & \
                              (cities_data['Wind Speed'] < 10) & (cities_data['Cloudiness'] ==0) ]
cities_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,kokopo,-4.35,152.26,26.93,75,0,3.18,PG,1595453500
33,matay,28.42,30.78,25.15,61,0,5.35,EG,1595453505
49,gat,31.61,34.76,26.67,63,0,0.45,IL,1595453342
51,vila,42.03,-8.16,25.00,64,0,1.50,PT,1595453512
55,arraial do cabo,-22.97,-42.03,22.00,94,0,3.10,BR,1595453512
79,bayan,46.08,127.40,21.24,75,0,2.23,CN,1595453516
104,haifa,32.82,34.99,24.44,86,0,2.34,IL,1595453521
134,vila velha,-20.33,-40.29,25.56,73,0,5.10,BR,1595453369
168,jabinyanah,35.03,10.91,26.00,54,0,0.26,TN,1595453538
210,honggang,46.47,124.87,23.31,76,0,3.00,CN,1595453549


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create a space in the data frame for the nearest hotel
cities_data["Hotel Name"] = ""
cities_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,kokopo,-4.35,152.26,26.93,75,0,3.18,PG,1595453500,
33,matay,28.42,30.78,25.15,61,0,5.35,EG,1595453505,
49,gat,31.61,34.76,26.67,63,0,0.45,IL,1595453342,
51,vila,42.03,-8.16,25.00,64,0,1.50,PT,1595453512,
55,arraial do cabo,-22.97,-42.03,22.00,94,0,3.10,BR,1595453512,


In [6]:
# Initial count...we will only be getting 10 hotels
count = 1

# Set up the parameters need for google's near by search
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Go through the rows in the data frame
for index, row in cities_data.iterrows():
    
    # We are only looking at 10 cities, so break if we've exceed that
    if count > 10:
        break
  
    # Get the longitude and latitude and add to params
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # Start building the url for the api
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Reguest the data
    hotels = requests.get(base_url, params=params)
    hotels = hotels.json()
    
    # Try to get a hotels, if one is found, increase counter
    try:
        cities_data.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        count +=1
    except (KeyError, IndexError):
        pass

In [7]:
# Create a dataframe with the 10 cities that have hotels
cities_10_data = cities_data.loc[cities_data['Hotel Name'] != '']
cities_10_data.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,kokopo,-4.35,152.26,26.93,75,0,3.18,PG,1595453500,Kokopo Beach Bungalow Resort
33,matay,28.42,30.78,25.15,61,0,5.35,EG,1595453505,Nag Aespen
49,gat,31.61,34.76,26.67,63,0,0.45,IL,1595453342,Desert Gat
51,vila,42.03,-8.16,25.00,64,0,1.50,PT,1595453512,Hotel Castrum Villae
55,arraial do cabo,-22.97,-42.03,22.00,94,0,3.10,BR,1595453512,Hotel da Canoa
79,bayan,46.08,127.40,21.24,75,0,2.23,CN,1595453516,Longfucheng Hotel
104,haifa,32.82,34.99,24.44,86,0,2.34,IL,1595453521,Bay Club Hotel
134,vila velha,-20.33,-40.29,25.56,73,0,5.10,BR,1595453369,Quality Suites Vila Velha
224,marsa matruh,31.35,27.25,24.14,74,0,4.33,EG,1595453554,Belle Vue Hotel Marsa Matrouh
232,faro,37.02,-7.93,25.00,78,0,4.60,PT,1595453322,Hotel Faro & Beach Club


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cities_10_data.iterrows()]
locations = cities_10_data[["Lat", "Lng"]]

In [9]:
# Create the heatmap again with the humity as weight and the 10 cities we have
humidity = cities_10_data["Humidity"].astype(float)

gmaps.configure(api_key=g_key)

# Plot Heatmap
fig = gmaps.figure(zoom_level=2, center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Add the markers for the 10 cities
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))